In [3]:
from datasets import load_dataset
dataset = load_dataset("Shannnh/baseline-dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/397 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4212 [00:00<?, ? examples/s]

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers', 'predicted_answer'],
        num_rows: 4212
    })
})


In [5]:
data = dataset["train"]

In [6]:
def calculate_f1(predicted, ground_truth):
    pred_tokens = predicted.split()
    gt_tokens = ground_truth.split()
    common = set(pred_tokens) & set(gt_tokens)
    if len(common) == 0:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gt_tokens)
    return 2 * precision * recall / (precision + recall)

f1_scores = [calculate_f1(pred, gt) for pred, gt in zip(data["predicted_answer"], data["answers"])]
average_f1 = sum(f1_scores) / len(f1_scores)
print(f"Average F1 Score: {average_f1:.4f}")

Average F1 Score: 0.3934


In [7]:
def calculate_exact_match(predicted, ground_truth):
    return int(predicted.strip() == ground_truth.strip())

em_scores = [calculate_exact_match(pred, gt) for pred, gt in zip(data["predicted_answer"], data["answers"])]
average_em = sum(em_scores) / len(em_scores)
print(f"Exact Match Score: {average_em:.4f}")

Exact Match Score: 0.1968


In [9]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

def calculate_q_bleu(predicted, ground_truth, question):
    # Tokenize inputs
    ref_tokens = ground_truth.split()
    hyp_tokens = predicted.split()
    question_tokens = question.split()

    reference = [ref_tokens]
    hypothesis = hyp_tokens
    bleu_score = sentence_bleu(reference, hypothesis)

    stop_words = set(stopwords.words('english'))
    important_ref_tokens = [token for token in ref_tokens if token.lower() not in stop_words]
    important_hyp_tokens = [token for token in hyp_tokens if token.lower() not in stop_words]

    key_match = len(set(important_ref_tokens) & set(important_hyp_tokens)) / max(len(set(important_ref_tokens)), 1)

    question_match = len(set(question_tokens) & set(hyp_tokens)) / max(len(set(question_tokens)), 1)

    q_bleu = 0.7 * bleu_score + 0.2 * key_match + 0.1 * question_match

    return q_bleu

data = {
    "predicted_answer": ["The capital of France is Paris.", "Python is a programming language."],
    "answers": ["Paris is the capital of France.", "Python is a widely used programming language."],
    "questions": ["What is the capital of France?", "What is Python?"]
}

q_bleu_scores = [
    calculate_q_bleu(pred, gt, q)
    for pred, gt, q in zip(data["predicted_answer"], data["answers"], data["questions"])
]
average_q_bleu = sum(q_bleu_scores) / len(q_bleu_scores)
print(f"Average Q-BLEU Score: {average_q_bleu:.4f}")


Average Q-BLEU Score: 0.1350


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
